In [1]:
import numpy as np

In [2]:
commande = {
    # switch
        # mode vol / grappin
    'state_switch_1': ..., # 0 --> position haut du switch 1 / 1 --> position basse du switch 1
    'state_switch_2': ..., # 0 --> position haut du switch 2 / 1 --> position basse du switch 2
    'state_switch_3': ..., # 0 --> position haut du switch 3 / 1 --> position milieu du switch 3 / 2 --> position basse du switch 3
    'state_switch_4': ..., # 0 --> position haut du switch 4 / 1 --> position basse du switch 4

    # position slider 
    'slider': ..., # valeur entre 0 et 100

    # position joystick
    'joystick': [..., ...], # [channel_1, channel_2]

    # potentiomètre 1
    'potentiometre_1': ..., # modification de la vitesse de rotation des moteurs grappin
}

In [ ]:
def GetCommMotors(a, b):
    r = np.abs(a) + np.abs(b)

    Fv_1 = -a if r <= 1 else -a/r
    Fv_2 = a if r <= 1 else a/r
    Fp_1 = b if r <= 1 else b/r
    Fp_2 = b if r <= 1 else b/r

    F_1 = Fv_1 + Fp_1
    F_2 = Fv_2 + Fp_2

    angle_1 = 0 if F_1 >= 0 else 180
    angle_2 = 0 if F_2 >= 0 else 180

    return abs(F_1), abs(F_2), angle_1, angle_2

In [ ]:
def map_linear(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)

def GetSpeedGrappin(ch3):
    
    DETECTION_ZONE = 0.5    # 0 <= x <= 2
    MAX_SPEED = 1000
    
    if ch3 < -DETECTION_ZONE/2:
        speedGrappin = map_linear(ch3, -1, -DETECTION_ZONE/2, -MAX_SPEED, 0)

    elif (-DETECTION_ZONE/2 <= ch3 and ch3 <= DETECTION_ZONE/2):
        speedGrappin = 0

    elif (DETECTION_ZONE/2 < ch3):
        speedGrappin = map_linear(ch3, DETECTION_ZONE/2, 1, 0, MAX_SPEED)
    
    return speedGrappin

In [ ]:
def FlightControl(commande) -> dict:

    speed_motor_G, speed_motor_D, angle_servomot_G, angle_servomot_D = GetCommMotors(commande['joystick'][0], commande['joystick'][1])
    speed_stepmotor = GetSpeedGrappin(commande['slider']) # valeur entre -1 et 1

    instruction_motor = {

        # gestion moteur avant droit
        'speed_AvD': speed_motor_D,
        'angle_AvD': angle_servomot_D,

        # moteur avant gauche
        'speed_AvG': speed_motor_G,
        'angle_AvG': angle_servomot_G,

        # moteur arrière droit
        'speed_ArD': speed_motor_D,
        'angle_ArD': angle_servomot_D,

        # moteur arrière gauche
        'speed_ArG': speed_motor_G,
        'angle_ArG': angle_servomot_G

    }

    instruction_grappin = {
        # control des vitesses de rotation des moteurs pas à pas
        'speed_stepmotor' : speed_stepmotor
    }
    

    control = {
        'instruction_motor': instruction_motor,
        'instruction_grappin': instruction_grappin,
        'instruction_ballast': ...,
    }


    return control